In [1]:
#importing required libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F
from torch.autograd import Variable

[!img]segnet.png

In [16]:
class SegNet(nn.Module):
    def __init__(self):
        super(SegNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64,64, kernel_size=3, padding=1)
                
        self.conv3 = nn.Conv2d(64,128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128,128, kernel_size=3, padding=1)
        
        self.conv5 = nn.Conv2d(128,256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(256,256, kernel_size=3, padding=1)
        
        self.conv7 = nn.Conv2d(256,512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        
        self.conv8 = nn.Conv2d(512,512, kernel_size=3, padding=1)        
        
        self.upSample = nn.MaxUnpool2d(2, stride=2)
      
        self.decode1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.decode2 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.dec_bn1 = nn.BatchNorm2d(512)
        self.dec_relu = nn.ReLU(inplace=True)
        
        self.dec_bn2 = nn.BatchNorm2d(256)
        
        self.decode3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.decode4 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.dec_bn3 = nn.BatchNorm2d(128)
        
        self.decode5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.decode6 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.dec_bn4 = nn.BatchNorm2d(64)
        self.decode7 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.decode8 = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        
        
    def forward(self, x):
        out= self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn1(out)
        out = self.relu(out)
        x1p, id1 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
        
        out = self.conv3(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv4(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.maxPool(out)
        x2p, id2 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
                
        out = self.conv5(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv6(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv6(out)
        out = self.bn3(out)
        out = self.relu(out)
        x3p, id3 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
        
        out = self.conv7(out)
        out = self.bn4(out)
        out = self.relu(out)
        out = self.conv8(out)
        out = self.bn4(out)
        out = self.relu(out)
        out = self.conv8(out)
        out = self.bn4(out)
        out = self.relu(out)
        x4p, id4 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
        
        out = self.conv8(out)
        out = self.bn5(out)
        out = self.relu(out)
        out = self.conv8(out)
        out = self.bn5(out)
        out = self.relu(out)
        out = self.conv8(out)
        out = self.bn5(out)
        out = self.relu(out)
        x5p, id5 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
        
        out = self.upSample(x5p, id5)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        
        out = self.upSample(out, id4)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        out = self.decode2(out)
        out = self.dec_bn2(out)
        out = self.dec_relu(out)
        
        out = self.upSample(out, id3)
        out = self.decode3(out)
        out = self.dec_bn2(out)
        out = self.dec_relu(out)
        out = self.decode3(out)
        out = self.dec_bn2(out)
        out = self.dec_relu(out)
        out = self.decode4(out)
        out = self.dec_bn3(out)
        out = self.dec_relu(out)
        
        out = self.upSample(out, id2)
        out = self.decode5(out)
        out = self.dec_bn3(out)
        out = self.dec_relu(out)
        out = self.decode6(out)
        out = self.dec_bn4(out)
        out = self.dec_relu(out)
        
        out = self.upSample(out, id1)
        out = self.decode7(out)
        out = self.dec_bn4(out)
        out = self.dec_relu(out)
        out = self.decode8(out)

        return out

In [17]:
model = SegNet()

In [18]:
model

SegNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv8): Conv2d(512, 512, kernel_size=(3, 3), stride=(1

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

import numpy as np
import tqdm

### Data preparation w/ VOC2012

In [20]:
train_img=[]
train_mask=[]
trainval_img=[]
trainval_mask=[]
val_img=[]
val_mask=[]

In [9]:
from torch.utils.data.dataset import Dataset  # For custom data-sets
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

PATH = "../VOC_2011_TrainVal/VOCdevkit/VOC2011/"
#train = open(PATH +"../Data/ImageSets/Segmentation/train.txt", 'r')
train = open(PATH +"ImageSets/Segmentation/train.txt", 'r')
lines = train.readlines()
for i, line in enumerate(lines):
    train_img.append(PATH +"JPEGImages/"+line.split('\n')[0]+".jpg")
    train_mask.append(PATH + "SegmentationClass/" + line.split('\n')[0] + ".png")
train.close()

trainval = open(PATH +"ImageSets/Segmentation/trainval.txt", 'r')
lines = trainval.readlines()
for i, line in enumerate(lines):
    train_img.append(PATH +"JPEGImages/"+line.split('\n')[0]+".jpg")
    train_mask.append(PATH + "SegmentationClass/"+line.split('\n')[0]+".png")
trainval.close()

val = open(PATH +"ImageSets/Segmentation/val.txt", 'r')
lines = val.readlines()
for i, line in enumerate(lines):
    val_img.append(PATH +"JPEGImages/"+line.split('\n')[0]+".jpg")
    val_mask.append(PATH +"SegmentationClass/"+line.split('\n')[0]+".png")
val.close()

In [10]:
from PIL import Image



class CustomDataset(Dataset):
    def __init__(self, image_paths, target_paths, train=True):   # initial logic happens like transform
        self.image_paths = image_paths
        self.target_paths = target_paths
        self.transforms = transforms.ToTensor()
        
    def transform(self, image, mask):
        resize=transforms.Resize(size=(520,520))
        image=resize(image)
        mask=resize(mask)
        
        # Random crop
        i, j, h, w = transforms.RandomCrop.get_params(
            image, output_size=(512, 512))
        image = TF.crop(image, i, j, h, w)
        mask = TF.crop(mask, i, j, h, w)

        # Random horizontal flipping
        num=random.random()
        if num > 0.5:
            image = TF.hflip(image)
            mask = TF.hflip(mask)

        # Random vertical flipping
        if num > 0.5:
            image = TF.vflip(image)
            mask = TF.vflip(mask)

         # Transform to tensor
        image = TF.to_tensor(image)
        mask = TF.to_tensor(mask)

        mean=[0.4546, 0.4405, 0.4051]
        std=[0.2322, 0.2282, 0.2337]
        image[0]=(image[0]-mean[0])/std[0]
        image[1]=(image[1]-mean[1])/std[1]
        image[2]=(image[2]-mean[2])/std[2]
        
        return image, mask
    
    

    def __getitem__(self, index):
        
        image = Image.open(self.image_paths[index]).convert("RGB")
        mask = Image.open(self.target_paths[index]).convert("P")

        image, label=self.transform(image, mask)
        
        label_size=label.size
        
        # One-hot encoding 
        h = label.shape[1]
        w= label.shape[2]
        target = torch.zeros(21, h, w)
        
        label.reshape((-1,2))
        for c in range(21):
            target[c][label[0] == c] = 1

        return {'x': image, 'y':target, 'l':label[0]} 

        

    def __len__(self):  # return count of sample we have
        return len(self.image_paths)


train_dataset = CustomDataset(train_img, train_mask, train=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

trainval_dataset = CustomDataset(trainval_img, trainval_mask, train=False)
trainval_loader = torch.utils.data.DataLoader(trainval_dataset, batch_size=8, shuffle=False)

val_dataset = CustomDataset(val_img, val_mask, train=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False)

### Train

In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")          
    
import random
for i, batch in enumerate(train_loader):
    print(i, batch['x'].shape, batch['y'].shape, batch['l'].shape)
    break

0 torch.Size([8, 3, 512, 512]) torch.Size([8, 21, 512, 512]) torch.Size([8, 512, 512])


In [22]:
best = 0
meaniou=100
epochs=100
model = model.to(device)
for epoch in range(epochs):
    for i, batch in enumerate(train_loader):
        inputs, labels = batch['x'].to(device), batch['y'].to(device)
        optimizer.zero_grad()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % print_every == 0:
            model.eval()
            with torch.no_grad():
                #paccuracy, meaniou = validation(model, validloader, criterion)
                print("Epoch: {}/{}.. ".format(epoch+1, epochs),
                      "Training loss: {:.5f}.. ".format(loss.item()))
            if best < meaniou:
                torch.save(model.state_dict(), "./segnet.pth")
                best = meaniou
            model.train()

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 2.00 GiB total capacity; 1.26 GiB already allocated; 11.00 MiB free; 17.19 MiB cached)